# Рендеринг видеоролика

Запустите все ячейки, чтобы отрендерить 20-секундное видео разрешением 640 
на 480 и частотой 24 кадра в секунду

**ВАЖНО! Должны быть установлены компилятор nvcc и opencv**


Создаём файл с конфигурационными параметрами

In [ ]:
file = open('.configs.txt', 'w')
file.write('480')
file.write('.res/%d.data')
file.write('640 480 90')
file.write('7 3 0 2 1 2 6 1 0 0')
file.write('2 0 0 0.5 0.1 1 4 1 0 0')
file.write('0 -2 0 1 0 0 1')
file.write('0 0 0 0 1 0 1')
file.write('0 2 0 0 0 1 1')
file.write('-5 -5 -1 -5 5 -1 5 5 -1 5 -5 -1 1 1 1')
file.write('10 0 15 0.294118 0.196078 0.0980392 4')
file.close()


Создаём папку для кадров

In [ ]:
!mkdir .res

In [ ]:
!mkdir .res_png

Запускаем рендеринг

In [ ]:
!nvcc --std=c++11 main.cu -o main && ./main < .configs.txt

Конвертируем полученные кадры в формат png.

In [ ]:
from PIL import Image
import struct
import ctypes

def from_data_to_png(png_path, data_path):
    fin = open(data_path, 'rb')
    (w, h) = struct.unpack('hi', fin.read(8))
    buff = ctypes.create_string_buffer(4 * w * h)
    fin.readinto(buff)
    fin.close()
    img = Image.new('RGBA', (w, h))
    pix = img.load()
    offset = 0
    for j in range(h):
        for i in range(w):
            (r, g, b, a) = struct.unpack_from('cccc', buff, offset)
            pix[i, j] = (ord(r), ord(g), ord(b), ord(a))
            offset += 4
    img.save(png_path)

for i in range(480):
    from_data_to_png('.res_png/'+str(i)+'.png', '.res/'+str(i)+'.data')


Создаём видео из полученных кадров

In [ ]:
import cv2
import os

input_directory = '.res_png'
output_video = 'video.mp4'
frame_rate = 24

image_files = [f for f in os.listdir(input_directory) if f.endswith('.png')]
image_files.sort(key=lambda x: int(x[:-4]))

image_path = os.path.join(input_directory, image_files[0])
img = cv2.imread(image_path)
height, width, _ = img.shape
video = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

for image_file in image_files:
    image_path = os.path.join(input_directory, image_file)
    img = cv2.imread(image_path)
    video.write(img)

cv2.destroyAllWindows()
video.release()


Удаляем ненужные файлы

In [ ]:
!rm -rf .res

In [ ]:
!rm -rf .res_png

In [ ]:
!rm .configs.txt